<a href="https://colab.research.google.com/github/etuckerman/surf_NLP/blob/main/surf_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Log into HuggingFace**

In [51]:
from huggingface_hub import notebook_login

notebook_login()